# CODE-DE/EO-Lab tutorial

<div style="text-align: right"><i> Beginner </i></div>

<center><h1> CODE-DE/EO-LAB Datenprozessierung und Zonale Statistiken vom Jupyter-Lab über BOTO3 mit API.  </h1></center>
Wir verwenden den Zugang vom Jupyter-Lab um auf Sentinel-2 Daten zugreifen zu können. Es wird die Such-API vom Data Explorer verwendet: https://explore.code-de.org/search.

Das Skript iteriert über eine Zeitreihe von Daten, berechnet den NDVI und Zonale Statistiken über ein Polygon und schreibt die Ergebnisse in eine *.csv Datei. 
<br>


**General Note 1**: Ausführung der Zellen durch pressen des <button class="btn btn-default btn-xs"><i class="icon-play fa fa-play"></i></button> button vom top MENU (oder `Shift` + `Enter`).
<br>
**General Note 2**: Falls der Kern nich mehr arbeitet, im the top MENU, klicke <button class="btn btn-default btn-xs"><i class="fa fa-repeat icon-repeat"></i></button> button. Dann, im top MENU, clicke  "Run" aund wähle "Run All Above Selected Cell".
<br>
**General Note 3**: Schauen Sie sich im [**FORUM**](https://forum.code-de.org/de/) um oder Kontaktieren Sie den Support! 
<br>

In [ ]:
!pip install boto3
!pip install rasterio
!pip install matplotlib
!pip install rasterstats
!pip install geopandas
# import libaries
import json
import requests
import rasterio
from rasterio import plot
import boto3
import botocore
import numpy as np
import geopandas as gpd
import matplotlib.pyplot as plt
from rasterstats import zonal_stats
import os
import csv

In [ ]:
#function to check credentials
def checkCredentials():
   # access CODE-DE / EO-Lab S3 endpoint (setup the keys in the "S3 access credentials" section in your profile in the Tenant Manager)
   access_key='<your access key goes here>'
   secret_key='<your secret key goes here>'
   host='http://data.cloud.code-de.org'
   s3=boto3.client('s3',aws_access_key_id=access_key, aws_secret_access_key=secret_key, endpoint_url=host,)
   print('Connecting to the server')
   
   return s3

In [ ]:
#function to search scenes using the datahub API, returning a file-list to process
def searchScenes(num):
  # here we provide an examplary API request - you can create your own in https://explore.code-de.org/search
  api_url="https://datahub.code-de.org/odata/v1/Products?$filter=((Attributes/OData.CSC.DoubleAttribute/any(i0:i0/Name%20eq%20%27cloudCover%27%20and%20i0/Value%20le%205))%20and%20(ContentDate/Start%20ge%202023-07-16T00:00:00.000Z%20and%20ContentDate/Start%20le%202024-07-22T23:59:59.999Z)%20and%20(Online%20eq%20true)%20and%20(OData.CSC.Intersects(Footprint=geography%27SRID=4326;POINT%20(7.112461%2050.737838)%27))%20and%20(((((Collection/Name%20eq%20%27SENTINEL-2%27)%20and%20(((Attributes/OData.CSC.StringAttribute/any(i0:i0/Name%20eq%20%27productType%27%20and%20i0/Value%20eq%20%27L2A%27)))))))))&$expand=Attributes&$expand=Assets&$orderby=ContentDate/Start%20asc&$top=20"    
  # Make a GET request to the API
  response = requests.get(api_url)
  
  # Check if the request was successful (status code 200)
  if response.status_code == 200:
      # Parse the JSON data from the response
      data = response.json()      
  else:
      print("Failed to retrieve data. Status code:", response.status_code)
      print("Response content:", response.text)
  
  flist=[]
  j=0
  ListLenght = len(data['value'])
  for i in range(ListLenght):
    d = data['value'][i]['S3Path']
    if j < num:   
      if "T32" in d: # we are only interested in Zone 32
        flist.append(d)
        j=j+1
    
  # now we have our list of files with num elements to iterate and calculate NDVI
  return flist

In [ ]:
# download files locally to your folder
def downloadFiles(S2Image,s3):
  #print(S2Image)
  prefix=S2Image[8:]
  print('Found files')
  res = s3.list_objects_v2(Delimiter='/', Bucket="CODEDE", Prefix=prefix)
  tmp=res['CommonPrefixes']
  # need to do this twice to ge the right pathname from within the *.safe file 
  print(tmp[0]['Prefix'])
  prefix=tmp[0]['Prefix']+'GRANULE/'
  res = s3.list_objects_v2(Delimiter='/', Bucket="CODEDE", Prefix=prefix)
  prefix=res['CommonPrefixes'][0]['Prefix']+"IMG_DATA/R10m/"
  res = s3.list_objects_v2(Delimiter='/', Bucket="CODEDE", Prefix=prefix)
  
  fileList=[]
  i=0
  if 'Contents' in res:
        print("Dateien im Bucket..")
        for obj in res['Contents']:
            #print(obj['Key'])
            fileList.append(obj['Key'])
  else:
        print("Keine Dateien im Bucket gefunden.") 
  for content in s3.list_objects(Delimiter='/', Bucket="CODEDE", Prefix=prefix)['Contents']:
         if content['Key'].endswith('.jp2'): # filter by .jp2 suffix
                object_name = content['Key']
                file_name = object_name.split('/')[-1] # assume last part as a target filename
                if i in [4,5]: # [2,3,4,5]
                  with open(file_name, 'wb') as data:
                    s3.download_fileobj('CODEDE',object_name, data) 
                  if i == 4:
                    bandRED=file_name
                  if i == 5:
                    bandNIR=file_name
         i=i+1    
  
  return (bandNIR, bandRED)


In [ ]:
# decide if you need a function for cloudmasking

In [ ]:
# calculate NDVI
def calcNDVI(bandNIR, bandRED):
    #open raster data using rasterio
    #print(bandNIR)
    with rasterio.open(bandNIR) as src:
      out_meta = src.meta
        
    src.close()  
    win = rasterio.windows.Window(6000, 7000, 2500, 2500) # we only read a subset around our polygon
    with rasterio.open(bandNIR) as src:
      out_meta = src.meta
      # Read the data from the window
      NIR = src.read(window=win) 
     
    RED = rasterio.open(bandRED, driver='JP2OpenJPEG').read(1,window=win)
    #read the transformation for the image subset
    transform = src.window_transform(win)    
    NDVI=(NIR*1.0-RED)/(NIR+RED)
    NDVI[(NDVI >1) | (NDVI < -1)] = np.nan
    
    #write NDVi image to file with the updated meta-information
    out_meta.update({"driver": "GTiff","height": win.height,
        "width": win.width, "dtype" : rasterio.float32, "transform": transform})

    with rasterio.open("ndvi.tif", "w", **out_meta) as dest:
      dest.write(NDVI)#, indexes = 1
    os.remove(bandNIR) # cleaning up
    os.remove(bandRED)
    return NDVI

In [ ]:
#main program
s3=checkCredentials()
# we want three images in our timeseries
flist = searchScenes(3)

#looping through the API search
i=0
row=[]
row.append(["Image","date","min","max","mean","median","std"])
vector_file  = "StadtBonnUTM32.geojson"
#Read the vector file using Geopandas
vector_data = gpd.read_file(vector_file)

for f in flist: 
  S2Image=f
  date=f[49:57]
  #use our download function
  (bandNIR,bandRED)=downloadFiles(S2Image,s3)
  #use our NDVI function
  NDVI=calcNDVI(bandNIR,bandRED)
  #setting plot size and reading data from file  
  fig, ax = plt.subplots(figsize=(10, 10))
  with rasterio.open('ndvi.tif') as src:
    NDVI = rasterio.open('ndvi.tif',transform=src.transform).read(1)
      
  plot.show(NDVI, ax=ax,cmap=plt.get_cmap('jet'),vmin=-0.2, vmax=1,transform=src.transform, title = "Sentinel-2 NDVI " + date)
  
  # Plot the vector data on top of the raster
  vector_data.plot(ax=ax, facecolor='none', edgecolor='red')

  # Display the plot
  plt.show()
  plt.close()
  src.close()
  # use your on geojson file for the stats
  res=zonal_stats(vector_file, "ndvi.tif",
            stats="count min mean max median std",nodata=np.nan)
  i=i+1
  print(i)
  row.append([i, date,res[0]['min'],res[0]['max'],res[0]['mean'],res[0]['median'],res[0]['std']])
  os.remove('ndvi.tif') # deleting the image after we are done.

#write formatted output to a csv file
np.savetxt("ZonalStats.csv", row, delimiter =", ", fmt ='% s')

As an exercise: read the .csv file and plot the data by date and NDVI. 